# Focos Queimadas / SP (2019)

**1 - Contexto de negócios e definição do problema**

O tema da preservação do meio ambiente está desde sempre presente no nosso dia a dia, e é um tema cada vez mais importante, dada a constante exploração da natureza e suas consequências que ameaçam a sobrevivência da própria espécie humana. 

As queimadas são uns dos principais problemas, que pelo menos no Brasil, (ainda) são consideradas crimes contra a natureza, se não feitas com licença do orgão ambiental competente.

As queimadas devastam áreas inteiras de vegetação, matam e expulsam a fauna, prejudicam o solo, emitem gases poluentes e fumaça que fazem mal à saúde dos animais (humanos inclusos), além de aumentar a temperatura como um todo, contribuindo para o famigerado aquecimento global e efeito estufa.

Existem dois fatores principais para as queimadas: o natural e o humano. No primeiro, se deve a incidência de descargas elétricas em áreas de pastagens, e também atividades vulcânicas. No segundo caso, são provocadas pelo ação humana, deliberada ou não, como queimadas para limpeza, balões, bitucas de cigarro, entre outras causas.

Para quem pratica a atividade agropecuária, é comum que nas épocas de maior seca, seja aproveitado para executar as tais limpezas no solo para o plantio. No caso de países com uma economia desindustrializada e dependente da exportação de commodities agrícolas, há um aumento constante dessas atividades, e com isso uma expansão cada vez maior das fronteiras agrícolas. Para que isso possa ocorrer, taca-se fogo em tudo o que há pela frente, de forma criminosa, em hectares e mais hectares de natureza.

Uma das soluções passa nas opções que o país faz, em investir tão pouco numa econômia industrial, e tanto na exportação de produtos vindos da agropecuária. Outra solução _ que não resolve _ é o da exploração responsável, onde nesse caso são adotados sistemas de monitoramento e fiscalicação eficientes por parte do poder público, e leis severas que punem de fato quem faz os chamados desmatamentos ilegais. A questão é o que o país entende por desmatamento ilegal. Achar que apenas agir dentro da lei não é suficiente, no sentido de proteger o meio ambiente, se tais leis continuarem a ser tão permissivas.

Seja como for, há de pelo menos começarmos a fazer o mínimo, e ir elevando a consciência popular para as questões do meio ambiente.

Neste relatório técnico, o objetivo é através de algoritmos, criar previsões de queimadas.

**2 - Abordagem de Aprendizado de Máquina para tratar o problema (descrição da tarefa e algoritmos a serem utilizados)**

Pensar a partir da análise já iniciada abaixo, quais algoritmos serão utilizados. Acredito que análises preditivas são as mais adequadas, para criar um relatório para os usuários que será capaz de apontar quais locais estão mais propensos a terem queimadas, e em qual época, e precisam de mais atenção das autoridades e da sociedade civil.


**3 - Canvas Analítico**



**4 - Coleta de Dados**

DB Queimadas - Instituto Nacional de Pesquisas Espaciais

Satélite:
    
- Aqua: https://www.embrapa.br/satelites-de-monitoramento/missoes/aqua
- Sensor MODIS: https://mundogeo.com/2004/05/23/o-sensor-modis-a-bordo-das-plataformas-terra-e-acqua/

**5 - Processamento/Tratamento de Dados**

Processo já iniciado abaixo.

In [1]:
# bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime, timezone, timedelta, time, date
from pytz import timezone
import pytz

#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=True)

In [2]:
# importar dados

df = pd.read_csv('data/queimadas_focos_2019-01-01_2019-12-31.csv', delimiter=',')
df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2019/09/03 17:42:00,NPP-375,Brasil,SAO PAULO,GUAIRA,Cerrado,0,0.0,0.0,-20.37233,-48.43229,7.4
1,2019/09/03 17:42:00,NPP-375,Brasil,SAO PAULO,GUAIRA,Cerrado,0,0.0,0.0,-20.37030,-48.33683,64.2
2,2019/09/03 17:42:00,NPP-375,Brasil,SAO PAULO,GUAIRA,Cerrado,0,0.0,0.0,-20.36897,-48.33117,64.2
3,2019/09/03 17:42:00,NPP-375,Brasil,SAO PAULO,GUAIRA,Cerrado,0,0.0,0.0,-20.36388,-48.33862,64.2
4,2019/09/03 17:42:00,NPP-375,Brasil,SAO PAULO,GUAIRA,Cerrado,0,0.0,0.0,-20.36255,-48.33293,64.2


Das colunas que temos, vamos trabalhar com as seguintes:

- **datahora**: Horário de referência da passagem do satélite segundo o fuso horário de Greenwich (GMT)
- **satelite**: Nome do algorítmo utilizado e referencia ao satélite provedor da imagem
- **municipio**: Nome do município. Para o Brasil foi utilizado como referência o dado do IBGE 2000
- **bioma** Bioma: uma unidade biológica ou espaço geográfico cujas características específicas são definidas pelo macroclima, o solo, a altitude, dentre outros critérios
- **diasemchuva**: Número de dias sem chuva até a detecção do foco
- **precipitacao**: Valor da precipitação acumulada no dia até o momento da detecção do foco
- **riscofogo**: Valor do Risco de Fogo previsto para o dia da detecção do foco (Vai de 0 até 1, e o valor 777.7 significa que é valor inválido)
- **frp**: Fire Radiative Power (Energia Radiativa do Fogo, intensidade da queimada), valor em MW (megawatts)

### Tratar dados

Excluir colunas indesejadas e procurar por valores nulos e faltantes.

In [3]:
df.drop(['pais', 'estado'], axis=1, inplace=True)
df.describe()

,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
count,54970.000000,54970.000000,54321.000000,54970.000000,54970.000000,26598.000000
mean,4.788266,0.347480,2.011816,-21.954733,-48.610741,19.257486
std,5.807469,1.969839,33.647930,1.101065,1.521500,48.637202
min,0.000000,0.000000,0.000000,-25.184420,-53.093230,0.000000
25%,0.000000,0.000000,0.000000,-22.776817,-49.519275,2.900000
50%,3.000000,0.000000,0.880000,-22.005645,-48.545005,7.100000
75%,8.000000,0.000000,1.000000,-21.050000,-47.448000,17.300000
max,39.000000,76.380000,777.700000,-19.807990,-44.230430,1935.500000


Encontrado vazios (NA) nas colunas riscofogo e frp.

In [4]:
df.isna().sum()

datahora            0
satelite            0
municipio           0
bioma               0
diasemchuva         0
precipitacao        0
riscofogo         649
latitude            0
longitude           0
frp             28372
dtype: int64

No caso da coluna `riscofogo` como são poucos valores, vou zerar essas linhas. No caso do campo `frp` tem milhares de linhas vazias, mas, pelo que entendi desse campo, ele nem sempre consegue ser medido pelo satélite, porque pode ter sido uma queimada de baixa intensidade, portanto vamos mantê-lo tal como está.

In [5]:
# Zerando os valores riscofogo quando vazios

df.loc[df.riscofogo.isna(),  'riscofogo'] = 0
# df.loc[df.riscofogo.isna(),  'riscofogo']

**Trabalhando com as datas**

Primeiro, se sabe que o fuso horário dos satélites é o GMT (UTC). Como a análise será feita numa região específica, no caso o estado de São Paulo, pode-se converter o fuso do campo datahora para o de São Paulo (-3 horas em relação ao GMT).

In [6]:
# Converter a datahora com fuso de sp, para um objeto data

df['datahora'] = pd.to_datetime(df['datahora'], errors='coerce')

In [7]:
def converter_fuso_sp(row):
    """
    Recebe uma data, seta o timezone UTC
    e retorna a data com fuso horário de São Paulo (-3h)
    """
    #utc = pytz.utc
    dt = pytz.utc.localize(row.datahora)
    sp_dt = dt.astimezone(timezone('America/Sao_Paulo'))
    return sp_dt.strftime('%Y-%m-%d %H:%M:%S')

In [8]:
df['datahora_tz'] = df.apply(converter_fuso_sp, axis=1)

Depois será necessário separar data e hora, para facilitar trabalhar com ambos.

In [9]:
def converter_data_hora(df, coluna, tipo):
    """
    Recebe o dataframe, uma coluna de data e hora
    E retorna data ou hora
    """
    if tipo == 'data':
        return df[coluna].dt.date
    else:
        return df[coluna].dt.time

In [10]:
df['data'] = converter_data_hora(df, 'datahora', 'data')
df['hora'] = converter_data_hora(df, 'datahora', 'hora')

In [11]:
# Converter a datahora com fuso de sp, para um objeto data

df['datahora_tz'] = pd.to_datetime(df['datahora_tz'], errors='coerce')

E em seguida criar a coluna `periodo_dia` para classificar os períodos do dia, baseados nas horas: 

- *manhã* (6:00 às 11:59)
- *tarde* (12:00 às 17:59)
- *noite* (18:00 às 23:59)
- *madrugada* (00:00 às 05:59).

O porquê disso será entendido posteriormente.

In [12]:
def retornar_periodo(row):
    """
    Verificar dentre qual intervalo de horas está uma hora específica
    e classificar dentro de um período do dia
    """
    if time(6, 0, 0) <= row.datahora_tz.time() < time(12, 0, 0):
        return 'manha'
    elif time(12, 0, 0) <= row.datahora_tz.time() < time(18, 0, 0):
        return 'tarde'
    elif time(18, 0, 0) <= row.datahora_tz.time() <= time(23, 59, 59):
        return 'noite'
    elif time(0, 0, 0) <= row.datahora_tz.time() < time(6, 0, 0):
        return 'madrugada'

In [13]:
df['periodo_dia'] = df.apply(retornar_periodo, axis=1)

In [14]:
df.head(5)

,datahora,satelite,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp,datahora_tz,data,hora,periodo_dia
0,2019-09-03 17:42:00,NPP-375,GUAIRA,Cerrado,0,0.0,0.0,-20.37233,-48.43229,7.4,2019-09-03 14:42:00,2019-09-03,17:42:00,tarde
1,2019-09-03 17:42:00,NPP-375,GUAIRA,Cerrado,0,0.0,0.0,-20.37030,-48.33683,64.2,2019-09-03 14:42:00,2019-09-03,17:42:00,tarde
2,2019-09-03 17:42:00,NPP-375,GUAIRA,Cerrado,0,0.0,0.0,-20.36897,-48.33117,64.2,2019-09-03 14:42:00,2019-09-03,17:42:00,tarde
3,2019-09-03 17:42:00,NPP-375,GUAIRA,Cerrado,0,0.0,0.0,-20.36388,-48.33862,64.2,2019-09-03 14:42:00,2019-09-03,17:42:00,tarde
4,2019-09-03 17:42:00,NPP-375,GUAIRA,Cerrado,0,0.0,0.0,-20.36255,-48.33293,64.2,2019-09-03 14:42:00,2019-09-03,17:42:00,tarde


**Risco fogo**

Vamos adicionar a coluna `riscofogo_nivel` para classificar os níveis do risco de fogo, que o próprio instituto utiliza: 

- Mínimo, abaixo de 0,15; 
- Baixo, de 0,15 a 0,4; 
- Médio, de 0,4 a 0,7; 
- Alto, de 0,7 a 0,95 ; 
- Crítico, acima de 0,95 até 1.

In [15]:
def classificacar_risco_fogo(row):
    """
    Recebe o valor de riscofogo, e retorna sua classificação
    """
    if row.riscofogo >= 0 and row.riscofogo < 0.15:
        return 'minimo'
    elif row.riscofogo >= 0.15 and row.riscofogo < 0.4:
        return 'baixo'
    elif row.riscofogo >= 0.4 and row.riscofogo < 0.7:
        return 'medio'
    elif row.riscofogo >= 0.7 and row.riscofogo < 0.95:
        return 'alto'
    elif row.riscofogo >= 0.95 and row.riscofogo <= 1:
        return 'critico'
    else:
        return 'na'

In [16]:
df['riscofogo_nivel'] = df.apply(classificacar_risco_fogo, axis=1)

In [17]:
df.head(2)

,datahora,satelite,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp,datahora_tz,data,hora,periodo_dia,riscofogo_nivel
0,2019-09-03 17:42:00,NPP-375,GUAIRA,Cerrado,0,0.0,0.0,-20.37233,-48.43229,7.4,2019-09-03 14:42:00,2019-09-03,17:42:00,tarde,minimo
1,2019-09-03 17:42:00,NPP-375,GUAIRA,Cerrado,0,0.0,0.0,-20.37030,-48.33683,64.2,2019-09-03 14:42:00,2019-09-03,17:42:00,tarde,minimo


### Análise dos dados

<b style='color:#5566aa'>
O Estado de São Paulo possui 645 municípios (informação de 2020), vamos verificar quantos municípios tiveram focos de queimadas.
</b>

In [18]:
print('No ano de 2019, {} cidades do Estado de SP tiveram focos de queimadas, isso equivale a {}%.'.format(
    df['municipio'].nunique(), np.round(df['municipio'].nunique() / 645 * 100),2)
)

No ano de 2019, 638 cidades do Estado de SP tiveram focos de queimadas, isso equivale a 99.0%.


Antes de prosseguirmos, é necessário entendermos algo: os satélites indicam focos de queimadas, e isso quer dizer que uma mesma queimada pode ter vários focos em regiões próximas. 

Pense no seguinte caso: Num sábadão fim de tarde um tiozinho da vida intencionalmente colocou fogo num monte de capim perto de um pasto, e o fogo se alastrou pela área. O satélite monitora a área, e identifica 3 focos de queimadas nessa região, e todos estes focos são originários do local onde o tiozinho tacou fogo _ mas isso não tem como o satélite saber, só pessoalmente para a equipe de bombeiros traçar o caminho do fogo, e dizer que todos os focos são digamos de uma só origem.

E porque estou explicando isso? 

Digamos que na sua cidade tenha tido um alto número de focos, mas isso não quer dizer necessariamente que são "queimadas diferentes", mas talvez que uma mesma queimada originou vários focos. Claro que do ponto de vista da preservação da natureza, isso não faz a menor diferença, pois quanto mais focos, mais grave é o problema.

<b style='color:#5566aa'>
Quais as quantidades de focos de queimadas por cidade?
</b>

In [38]:
df.groupby(['municipio']).value_counts().reset_index()
#df_focos_cidade = df.groupby(['municipio']).value_counts().sort_values(ascending=False)
#df_focos_cidade.reset_index()

,municipio,datahora,satelite,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp,datahora_tz,data,hora,periodo_dia,riscofogo_nivel,0
0,ADAMANTINA,2019-07-17 17:42:00,NPP-375,Mata Atlantica,0,0.00,1.00,-21.69783,-51.10107,1.8,2019-07-17 14:42:00,2019-07-17,17:42:00,tarde,critico,1
1,ADAMANTINA,2019-07-30 16:54:00,NPP-375,Mata Atlantica,5,0.00,1.00,-21.70208,-51.06310,1.4,2019-07-30 13:54:00,2019-07-30,16:54:00,tarde,critico,1
2,ADAMANTINA,2019-11-05 16:55:00,AQUA_M-T,Mata Atlantica,0,0.00,0.00,-21.46000,-51.01800,34.5,2019-11-05 13:55:00,2019-11-05,16:55:00,tarde,minimo,1
3,ADAMANTINA,2019-11-05 16:18:00,NPP-375,Mata Atlantica,0,0.00,0.00,-21.45473,-51.02302,31.9,2019-11-05 13:18:00,2019-11-05,16:18:00,tarde,minimo,1
4,ADAMANTINA,2019-11-05 16:18:00,NPP-375,Mata Atlantica,0,0.00,0.00,-21.45511,-51.02711,35.8,2019-11-05 13:18:00,2019-11-05,16:18:00,tarde,minimo,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26593,ZACARIAS,2019-07-17 05:12:00,NPP-375,Mata Atlantica,0,0.40,1.00,-21.12046,-50.05965,6.8,2019-07-17 02:12:00,2019-07-17,05:12:00,madrugada,critico,1
26594,ZACARIAS,2019-07-17 05:12:00,NPP-375,Mata Atlantica,0,0.40,1.00,-21.12085,-50.05544,6.8,2019-07-17 02:12:00,2019-07-17,05:12:00,madrugada,critico,1
26595,ZACARIAS,2019-07-17 05:12:00,NPP-375,Mata Atlantica,0,0.40,1.00,-21.12125,-50.05118,2.8,2019-07-17 02:12:00,2019-07-17,05:12:00,madrugada,critico,1
26596,ZACARIAS,2019-07-17 03:30:00,NPP-375,Mata Atlantica,0,0.40,1.00,-21.10876,-50.07942,5.3,2019-07-17 00:30:00,2019-07-17,03:30:00,madrugada,critico,1


In [35]:
df_focos_cidade.reset_index(name = 'count')

,municipio,datahora,satelite,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp,datahora_tz,data,hora,periodo_dia,riscofogo_nivel,count
0,ADAMANTINA,2019-07-17 17:42:00,NPP-375,Mata Atlantica,0,0.00,1.00,-21.69783,-51.10107,1.8,2019-07-17 14:42:00,2019-07-17,17:42:00,tarde,critico,1
1,PATROCINIO PAULISTA,2019-09-18 03:48:00,NPP-375,Cerrado,11,0.00,1.00,-20.54280,-47.15476,1.8,2019-09-18 00:48:00,2019-09-18,03:48:00,madrugada,critico,1
2,PATROCINIO PAULISTA,2019-07-19 04:36:00,NPP-375,Cerrado,0,0.00,1.00,-20.58270,-47.32698,1.2,2019-07-19 01:36:00,2019-07-19,04:36:00,madrugada,critico,1
3,PATROCINIO PAULISTA,2019-07-19 04:36:00,NPP-375,Cerrado,0,0.00,1.00,-20.58230,-47.32515,1.1,2019-07-19 01:36:00,2019-07-19,04:36:00,madrugada,critico,1
4,PATROCINIO PAULISTA,2019-09-20 04:30:00,AQUA_M-M,Cerrado,12,5.11,1.00,-20.54300,-47.14400,29.9,2019-09-20 01:30:00,2019-09-20,04:30:00,madrugada,critico,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26593,GUAIRA,2019-09-20 04:54:00,NPP-375,Cerrado,18,0.44,1.00,-20.23411,-48.33631,1.8,2019-09-20 01:54:00,2019-09-20,04:54:00,madrugada,critico,1
26594,GUAIRA,2019-09-24 16:06:00,NPP-375,Cerrado,3,0.00,1.00,-20.27882,-48.31800,20.5,2019-09-24 13:06:00,2019-09-24,16:06:00,tarde,critico,1
26595,GUAIRA,2019-09-24 16:06:00,NPP-375,Cerrado,3,0.00,1.00,-20.27844,-48.31372,20.5,2019-09-24 13:06:00,2019-09-24,16:06:00,tarde,critico,1
26596,GUAIRA,2019-09-24 16:06:00,NPP-375,Cerrado,3,0.00,1.00,-20.27718,-48.31974,19.3,2019-09-24 13:06:00,2019-09-24,16:06:00,tarde,critico,1


Vamos aproveitar e obtermos a média, a mediana e o desvio padrão de focos de queimadas por cidade.

In [ ]:
print('Média:', round(np.mean(df_focos_cidade),2))
print('Mediana:', round(np.median(df_focos_cidade),2))
print('Desvio padrão:', round(np.std(df_focos_cidade),2))

Quanto maior o afastamento do desvio da padrão da média, sugere uma alta variabilidade dos dados e até a presença dos chamados `outliers`, ou seja, valores muito fora do padrão. A mediana menor que a média também sugere a presença dos outliers, e um gráfico com a "cauda" maior para a direita.  Vamos fazer um histograma, para vermos se isso confere.

### Por região

<b style='color:#5566aa'>
  Plotar um histograma para termos uma noção da distribuição da quantidade de focos de queimadas por cidade 
</b>

In [ ]:
df_focos_cidade.plot();

---

## Dúvidas

Retiradas do site do INPE algumas informações complementares sobre o assunto.

### Que tamanho de queimada os satélites detectam?

Para os satélites de órbita polar (NOAAs a 800 km acima da superfície, e TERRA e AQUA a 710 km), trabalhos de validação de campo indicam que uma frente de fogo com cerca de **30m de extensão por 1m de largura**, ou maior, será detectada.

Um foco de calor tão pequeno quanto uma frente de queimada de 30 metros de extensão por 1 m de largura pode ser registrado pelos instrumentos de observação instalados nos satélites. Atualmente, são usadas três gerações de sensores para essa finalidade: o AVHRR, mais antigo; o Modis, de tecnologia intermediária; e o VIIRS, mais moderno. Um instrumento desse último tipo – presente nos satélites Suomi NPP e NOAA-20, da National Oceanic and Atmospheric Administration (NOAA), dos Estados Unidos – detecta 10 vezes mais focos que um Modis a bordo dos satélites Aqua e Terra, ambos da Nasa. Nuvens de chuvas são obstáculos para o bom funcionamento dos sensores, mas não as nuvens de fumaça das queimadas.

#### Quais queimadas não são detectadas ? (Por que a queimada que eu vi não foi detectada ?)
As seguintes condições impedem ou prejudicam muito a detecção das queimadas:

Frentes de fogo com menos de 30 m;
Fogo apenas no chão de uma floresta densa, sem afetar a copa das árvores;
Nuvens cobrindo a região (atenção - nuvens de fumaça não atrapalham!);
Queimada de pequena duração, ocorrendo entre o horário das imagens disponíveis;
Fogo em uma encosta de montanha, enquanto que o satélite só observou o outro lado;
Imprecisão na localização do foco de queima, que no melhor caso é de cerca de 375 m, mas chegando a 6 km.

#### Desde quando o INPE faz este trabalho de detecção de queimadas? 

Este sistema teve início em meados de 1985 durante um experimento de campo conjunto entre pesquisadores do INPE e da NASA, e vem evoluindo continuamente desde 1987 quando passou a operar rotineiramente. Ele foi particularmente aperfeiçoado a partir de 1998 mediante apoio do programa nacional Proarco no Ibama, criado para controlar as queimadas e o desmatamento no arco do desmatamento da Amazônia, com recursos do Ministério do Meio Ambiente.

---

### Bioma

O bioma pode ser definido, segundo o IBGE, como "um conjunto de vida vegetal e animal, constituído pelo agrupamento de tipos de vegetação contíguos e que podem ser identificados a nível regional, com condições de geologia e clima semelhantes e que, historicamente, sofreram os mesmos processos de formação da paisagem, resultando em uma diversidade de flora e fauna própria." Esse termo provém do grego Bio = vida e Oma = grupo ou massa e foi utilizado pela primeira vez, segundo alguns autores, por Shelford. Para outros, o termo foi criado por Clements.

### Risco Fogo

O cálculo do `risco fogo` é um tanto complexo, e leva em consideração fatores como dias sem chuva, tipo da vegetação, elevação topográfica, umidade relativa do ar, temperatura máxima e presença de fogo na região. Sua escala vai de 0 até 1, e se divide nas seguintes categorias: Mínimo, abaixo de 0,15; Baixo, de 0,15 a 0,4; Médio, de 0,4 a 0,7; Alto, de 0,7 a 0,95 e; Crítico, acima de 0.95.

O número 777,7 representa *valor inválido*, normalmente relacionado como **área urbana ou corpos d'água**, onde não faz sentido calcular o Risco de Fogo.

### FRP

A Energia Radiativa do Fogo (FRP) é uma técnica para quantificar a biomassa queimada usando dados de sensoriamento remoto. A FRP mede a energia radiante emitida por unidade de tempo pela vegetação queimada.

Da revista FAPESP (link no fim da página):

Além de monitorar a localização dos focos de calor, os satélites registram a **intensidade da queimada**. Medida como FRP (sigla em inglês para potência radiativa do fogo) (...) "A potência radiativa do fogo é uma medida instantânea da energia emitida", (...) "O fogo de desmatamento costuma ter emissões mais energéticas porque é proveniente de madeira acumulada com grande concentração de biomassa. É diferente do que ocorre com o fogo em capim, que tem uma carga combustível menor."

### Megawatt

Um megawatt é de 1 milhão de watts (W) de potência. Os geradores de eletricidade são frequentemente medidos em megawatts de capacidade de geração de energia de pico. Então, um megawatt (MW) é equivalente a 1.000 quilowatts ou 1.000.000 watts. Para colocar isso em perspectiva, uma lâmpada doméstica costuma ter 100 watts, portanto, um megawatt equivale a muitas lâmpadas.


---

### Referências

- <a href="http://queimadas.dgi.inpe.br/queimadas/portal/informacoes/perguntas-frequentes" target="_blank">Programa Queimadas, do Instituto Nacional de Pesquisas Espaciais (Inpe)</a>
- <a href="https://revistapesquisa.fapesp.br/como-monitorar-o-fogo/" target="_blank">Pesquisa FAPESP - Como monitorar o fogo</a>
- <a href="https://www.scielo.br/scielo.php?pid=S0102-77862017000200255&script=sci_abstract&tlng=pt" target="_blank">Scielo - Revista Brasileira de Meteorologia</a>
- <a href="http://queimadas.cptec.inpe.br/~rqueimadas/documentos/RiscoFogo_Sucinto.pdf" target="_blank">INPE - MÉTODO DO CÁLCULO DO RISCO DE FOGO DO PROGRAMA DO INPE - VERSÃO 11, JUNHO/2019</a>
- <a href="https://brasilescola.uol.com.br/o-que-e/biologia/o-que-e-bioma.htm" target="_blank">Brasil Escola - O que é bioma?</a>
- <a href="https://brasilescola.uol.com.br/brasil/biomas-brasileiros.htm" target="_blank">Brasil Escola - Biomas brasileiros</a>
- <a href="https://www.portalsaofrancisco.com.br/fisica/megawatt" target="_blank">Portal São Francisco - Megawatt</a>
- <a href="https://pt.wikipedia.org/wiki/S%C3%A3o_Paulo_(estado)" target="_blank">Wikipedia - São Paulo (estado)</a>
- https://pynative.com/python-timezone/